In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = AdultDataset(device=device)

In [3]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_fb_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_fb_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_fb_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_fb_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_fb_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            try:
                train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            except:
                continue
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

# SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['delta_effort'] = 0.5

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.88epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.68epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  5.98epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.75epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val    ei          dp          eo        eodd
---------------  ---------  ----------------  --------------  ----  ----------  ----------  ----------
         0.0001          0          0.759625        0.759158     0  0.00732228  0.00195639  0.00195639
         0.001           0          0.837043        0.842533     0  0.119958    0.0720145   0.0720145
         0.01            0          0.846036        0.85082      0  0.14373     0.0879802   0.0879802
         0.1             0          0.844875        0.843362     0  0.102196    0.0600302   0.0600302


# EI Hyperparameter Selection

In [5]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [1e-3, 1e-2]
EI_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EI_hp_test['batch_size'] = 128
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:33<00:00,  3.01epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:35<00:00,  2.82epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:19<00:00,  5.17epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:20<00:00,  4.85epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val    ei           dp          eo        eodd
---------------  ---------  ----------------  --------------  ----  -----------  ----------  ----------
          0.001        0            0.846492        0.851318     0  0.130307     0.0882806   0.0882806
          0.001        0.2          0.844502        0.84966      0  0.0977385    0.00459637  0.034459
          0.001        0.5          0.812425        0.817504     0  0.044439     0.0405664   0.0405664
          0.001        0.8          0.751295        0.750207     0  0            0           0
          0.001        1            0.248829        0.249793     0  0.00066302   0.00132714  0.00132714
          0.001        2            0.248705        0.249793     0  0            0           0
          0.001       10            0.248705        0.249793     0  0            0           0
          0.01         0            0.847445        0.848997  

In [7]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [1e-3, 1e-2]
DP_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
DP_hp_test['batch_size'] = 128
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:22<00:00,  4.39epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:23<00:00,  4.30epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.06epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.14epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val    ei          dp          eo        eodd
---------------  ---------  ----------------  --------------  ----  ----------  ----------  ----------
          0.001        0            0.846533        0.851318     0  0.130307    0.0882806   0.0882806
          0.001        0.2          0.659497        0.645284     0  0.05405     0.0265657   0.0265657
          0.001        0.5          0.625347        0.617603     0  0.00487641  0.0378349   0.0575579
          0.001        0.8          0.685814        0.674623     0  0.0668411   0.00840045  0.00861271
          0.001        1            0.658129        0.651251     0  0.143403    0.00859495  0.134918
          0.001        2            0.401882        0.410575     0  0.0548202   0.0524278   0.117006
          0.001       10            0.48738         0.496934     0  0.0346733   0.0112807   0.0112807
          0.01         0            0.847445    

In [6]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [1e-3, 1e-2]
EO_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EO_hp_test['batch_size'] = 128
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:26<00:00,  3.84epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.98epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.02epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.98epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val    ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ----  ----------  ---------  ---------
          0.001        0            0.846699        0.851318     0  0.129628    0.0803234  0.0803234
          0.001        0.2          0.844668        0.850655     0  0.116599    0.0216404  0.0456275
          0.001        0.5          0.765386        0.766451     0  0.0158482   0.0378464  0.0378464
          0.001        0.8          0.751295        0.750207     0  0           0          0
          0.001        1            0.248705        0.249793     0  0           0          0
          0.001        2            0.248705        0.249793     0  0           0          0
          0.001       10            0.248705        0.249793     0  0           0          0
          0.01         0            0.847404        0.849494     0  0.129304    0.0793252 

In [5]:
EOD_hp_test = SGD_hp_test.copy()
EOD_hp_test['learning_rate'] = [1e-3, 1e-2]
EOD_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EOD_hp_test['batch_size'] = 128
EOD_hp_test['fairness'] = 'EOD'

hyperparameter_test(dataset, EOD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.58epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.77epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  6.05epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  6.05epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val    ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----  ---------  ---------  ---------
          0.001        0            0.837043        0.842533     0  0.119958   0.0720145  0.0720145
          0.001        0.2          0.837084        0.842533     0  0.119958   0.0720145  0.0720145
          0.001        0.5          0.837291        0.843362     0  0.119966   0.0726781  0.0726781
          0.001        0.8          0.836794        0.844522     0  0.119485   0.0763249  0.0763249
          0.001        1            0.751295        0.750207     0  0          0          0
          0.001        2            0.248581        0.249793     0  0          0          0
          0.001       10            0.248581        0.249793     0  0          0          0
          0.01         0            0.846036        0.85082      0  0.14373    0.0879802  0